In [135]:
# the following code allows you to import function from
# jupyter notebook in other directory

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [136]:
import functools
%run ../gw/gw_wrapped_compressed.ipynb
%run ../compressed_class_list/compressed_class_list.ipynb

IndexError: list index out of range

In [137]:
class ChiangOGWInvariant():
    def __init__(self, use_recursion_a=True, ogw_1_0_g2=var('x'), ogw_2_2=var('y'), debug=False):
        self.use_recursion_a = use_recursion_a
        self.ogw_1_0_g2 = ogw_1_0_g2
        #self.ogw_1_0_g2 = -27/32
        self.ogw_2_2 = ogw_2_2
        self.debug = debug

    @functools.lru_cache(maxsize=None)
    def chiang_ogw_invariant(self, n, b, k, A, layer=0):
        #print(f"b:{b}, k:{k}, A:{A}")
        if isinstance(A, list):
            A = ClassList.compress(A, n)

        l = len(A)      
        if A.invalid_class_list:
            return 0    

        # wall-crossing theorem (Elad)
        if l > 0 and A[0] == -1:
            return -self.chiang_ogw_invariant(n, b, k+1, A[:-1]) 

        #degree axiom (Elad)
        if 2*sum(A) != (n - 3 + 2*b + k + 2*l - k*n):  
            # The maslov index is twice the chern class n+1 for holomorphic spheres. 
            # Using the invoulotion we can deduce that its half of this for disks thus mu(b)=b*(n+1) 
            # p 595 mcduff 
            #ogw_d[str((n, b, k, A)) + ", degree axiom"] = 0
            if layer == 1:
                print("degree axiom fails")
            return 0    

        # other
        #if k == 0 and is_even(b):
        #    return 0    

        # zero axiom
        if b == 0:
            if k == 1 and l == 1 and A[0] == 0:
                return -1
            else:
                return 0    

        # unit axiom (May)
        # handle all beta = 0 cases
        if l > 0 and A[0] == 0:
            if b == 0 and k == 1 and l == 1:
                return -1
            else:
                return 0    

        # divisor axiom 
        if l > 0 and A[0] == 1:
            #A.pop(0)
            return (b/4)*self.chiang_ogw_invariant(n, b, k, A[1:])   

        # base cases
        if b == 1 and k == 1 and A == []:
            return 3    

        if b == 1 and k == 0 and A == [2]:
            return self.ogw_1_0_g2
        
        if b == 2 and k == 2 and A == []:
            return 2
        
        # TODO: hand check
        if b == 2 and k == 0 and A == [2, 2]:
            return (-1 + self.chiang_ogw_invariant(n, 2, 1, ClassList.compress([2], n))^2)\
                    /(self.chiang_ogw_invariant(n, 2, 2, ClassList.compress([], n)))
        
        # TODO : hand check
        if b == 2 and k == 1 and A == [2]:
            return -self.chiang_ogw_invariant(n, 1, 1, ClassList.compress([1], n))\
                    *self.chiang_ogw_invariant(n, 1, 1, ClassList.compress([1], n))
        
        if b == 2 and k == 0 and A == [3]:
            return 2*(self.chiang_ogw_invariant(n, 2, 0, ClassList.compress([2, 2], n)) \
                    + self.chiang_ogw_invariant(n, 1, 1, ClassList.compress([1], n))*\
                        self.chiang_ogw_invariant(n, 1, 0, ClassList.compress([1, 2], n)))
             

        # recursion (a)
        if l >= 2 and not (k >= 1 and not self.use_recursion_a):
            cor = self.chiang_ogw_invariant(n, b, k, [A[0]-1, A[1]+1] + A[2:])
            col = 0
            ool = 0
            oor = 0
            A_0 = A[0]
            A_1 = A[1]  

            for t in A[2:].bipart():
                I1 = t[0]
                I2 = t[1]
                bipart_coeff = ClassList.bipart_coeff(A[2:], I1)    

                for i in range(0, n+1):
            
                    L1_cor = [1, A_1, i] + I1
                    L2_cor = [n-i, A_0-1] + I2 
                    for b1 in range(1, b):
                        b_ = 0
                        if (b - b1) % 4 == 0:
                            b_ = (b - b1) / 4
                        else:
                            continue
                        
                        cor += bipart_coeff \
                            * (-1)^(b_*(n+1)/2) *  gw_invariant(n, b_, L1_cor) \
                            *  self.chiang_ogw_invariant(n, b1, k, L2_cor)   

                for i in range(0, n+1):
                
                    L1_col = [1, A[0] - 1, i] + I1 
                    L2_col = [n-i, A[1]] + I2 
                    for b1 in range(1, b):
                        b_ = 0
                        if (b - b1) % 4 == 0:
                            b_ = (b - b1) / 4
                        else:
                            continue
                        
                        col += bipart_coeff \
                            * (-1)^((n+1)*b_/2)* gw_invariant(n, b_, L1_col) \
                            * self.chiang_ogw_invariant(n, b1, k, L2_col)    

                L1_ool = [1, A[0]-1] + I1
                L2_ool = [A[1]] + I2
                for b1 in range(1, b):
                    b2 = b - b1 

                    for k1 in range(0, k+1):
                        k2 = k - k1
                        ool += bipart_coeff \
                            * binomial(k, k1)*self.chiang_ogw_invariant(n, b1, k1, L1_ool) \
                            * self.chiang_ogw_invariant(n, b2, k2 + 1, L2_ool)   

                L1_oor = [1, A[1]] + I1
                L2_oor = [A[0] - 1] + I2    
                for b1 in range(1, b):
                    b2 = b - b1 

                    for k1 in range(0, k+1):
                        k2 = k - k1 

                        oor += bipart_coeff \
                            * binomial(k, k1) * self.chiang_ogw_invariant(n, b1, k1, L1_oor) \
                            * self.chiang_ogw_invariant(n, b2, k2 + 1, L2_oor)   

            result = cor - col - oor + ool
            if self.debug:
                print(f"recursion a: {result}")
            return result
        
        # recursion (c)
        if l >= 1 and k >= 1:
            #print(f"used c, A:{A}")
            co = 0
            ool = 0
            oor = 0
            A_0 = A[0]  

            for t in A[1:].bipart():
                I1 = t[0]
                I2 = t[1]   

                bipart_coeff = ClassList.bipart_coeff(A[1:], I1)    

                for i in range(0, n+1):
                    for b1 in range(0, b): # make sure that b1 is never b
                        b_ = 0
                        if (b - b1) % 4 == 0:
                            b_ = (b - b1) / 4
                        else:
                            continue

                        co += bipart_coeff \
                            * gw_invariant(n, b_, [A_0 - 1, 1, i] + I1) \
                            * self.chiang_ogw_invariant(n, b1, k, [n - i] + I2)   

                for b1 in range(1, b+1): # zero when b1 = 0
                    b2 = b - b1 

                    for k1 in range(0, (k-1)+1):
                        k2 = k - 1 - k1
                        ool += bipart_coeff \
                            * binomial(k-1, k1) \
                            * self.chiang_ogw_invariant(n, b1, k1, [A_0 - 1, 1] + I1) \
                            * self.chiang_ogw_invariant(n, b2, k2 + 2, I2)   

                for b1 in range(1, b+1): # exclude b1 = 0, which is zero, but would still cause infinite recursion
                    b2 = b - b1

                    for k in range(0, (k-1)+1):
                        k2 = k - 1 - k1
                        oor += bipart_coeff \
                            * binomial(k-1, k1) \
                            * self.chiang_ogw_invariant(n, b1, k1 + 1, [A_0 - 1] + I1) \
                            * self.chiang_ogw_invariant(n, b2, k2 + 1, [1] + I2) 

            result =  -co + ool - oor
            if self.debug:
                print(f"recursion c : {result}")
            return result   

        
        # recursion (d)
        if k >= 2:
            sum1 = 0
            sum2 = 0
            sum3 = 0
            
            for t in A.bipart():
                I1 = t[0]
                I2 = t[1]   

                bipart_coeff = ClassList.bipart_coeff(A, I1)    

                for i in range(0, n+1):
                    for b1 in range(0, (b+2)+1): # make sure that b1 is never b
                        b_ = 0
                        if ((b + 2) - b1) % 4 == 0:
                            b_ = ((b + 2) - b1) / 4
                            if b_ == 0:
                                continue
                        else:
                            continue
                        sum1 += bipart_coeff \
                            * gw_invariant(n, b_, [2, 2, i] + I1) \
                            * self.chiang_ogw_invariant(n, b1, k-1, [n - i] + I2)
                
                for b1 in range(2, b+1): # zero when b1 = 0
                    b2 = (b + 2) - b1 

                    for k1 in range(0, (k-2)+1):
                        k2 = (k - 2) - k1
                        sum2 += bipart_coeff \
                            * binomial(k-2, k1) \
                            * self.chiang_ogw_invariant(n, b1, k1+1, [2] + I1) \
                            * self.chiang_ogw_invariant(n, b2, k2+1, [2] + I2)
                    
                for b1 in range(2, b+1):
                    b2 = (b + 2) - b1
                    for k1 in range(0, (k-2)+1):
                        k2 = (k - 2) - k1
                        
                        if b1 == b and k1 == k - 2:
                            continue
                            
                        sum3 += bipart_coeff \
                            * binomial(k-2, k1) \
                            * self.chiang_ogw_invariant(n, b1, k1+2, I1) \
                            * self.chiang_ogw_invariant(n, b2, k2, [2, 2] + I2)
            
            result = (sum1 - sum2 + sum3)/(self.chiang_ogw_invariant(n, 2, 0, ClassList.compress([2, 2], n)))       
            if self.debug:
                print(f"recursion d: {result}")
            return result   

        print(f"unexpected situation is , b: {b}, k: {k}, A:{A.c_list}, l: {l}")
        return 0

In [138]:
# get value that satisfies the degree axiom 
def get_l1(n, b, k, l2, A1, A2):
    m = n - 3 + 2*b + k + 2*l2 - k*n - 2*l2 * A2 # degree axiom
    if (m % (2*(A1 - 1))) == 0:
        return int(m / (2*(A1 - 1)))
    else:
        -1


In [139]:

chiang_ogw = ChiangOGWInvariant(debug=False)
w = chiang_ogw.chiang_ogw_invariant(3, 2, 0, ClassList.compress([2, 2], 3))
print(f"w:{w}")
#b == 2 and k == 0 and A == [2, 2]

w:-175/512


In [140]:
chiang_ogw = ChiangOGWInvariant(debug=False)
# k = 0, no boundary points
for b in range(0, 10):
    for l2 in [1, 3, 5]:
        A1 = 2
        A2 = 3
        l1 = get_l1(3, b, 0, l2, A1, A2)
        if l1 < 0 :
            continue
        A = [A1]*l1 + [A2]*l2
        chiang = chiang_ogw.chiang_ogw_invariant(3, b, 0, ClassList.compress(A, 3))
        print(f"b: {b}, A: {A}, chiang ogw: {chiang}")

b: 2, A: [3], chiang ogw: 3/8*x - 175/256
b: 3, A: [2, 3], chiang ogw: -9/64*x + 525/2048
b: 4, A: [2, 2, 3], chiang ogw: 1/32*x^2 + 135/1024*x - 7875/32768
b: 5, A: [2, 2, 2, 3], chiang ogw: 1/262144*(96*x - 175)*(32*x + 27) + 3/8192*(192*x - 175)*x - 5/16384*(96*x - 175)*x - 9/64*x^2 + 4197/16384*x + 174825/524288
b: 6, A: [2, 2, 2, 2, 3], chiang ogw: -3/2097152*(320*x - 323)*(96*x - 175) + 3/1048576*(192*x - 175)*(96*x - 175) - 57/8388608*(96*x - 175)*(32*x + 27) + 3/45875200*(64*(16*(32*x + 81)*x + 1575)*x - 856800*x + 289275)*x - 3/22937600*(64*(16*(32*x + 81)*x + 1575)*x - 91875)*x - 45/131072*(192*x - 175)*x + 27/262144*(192*x - 875)*x + 165/262144*(96*x - 175)*x + 81/256*x^2 - 127203/262144*x - 4985575/8388608
b: 6, A: [3, 3, 3], chiang ogw: 1/262144*(192*x - 175)*(96*x - 175) - 1/2867200*(64*(16*(32*x + 81)*x + 1575)*x - 91875)*x - 9/16384*(96*x - 175)*x + 9/128*x^2 - 525/4096*x - 14175/131072
b: 7, A: [2, 2, 2, 2, 2, 3], chiang ogw: -81/67108864*(192*x - 175)*(192*x - 875) + 

b: 9, A: [2, 2, 2, 2, 2, 2, 2, 3], chiang ogw: -3/150323855360*(64*(16*(32*x + 81)*x + 1575)*x - 856800*x + 289275)*(32*(32*x + 27)*x - 4096*x^2 - 22464*x + 61425) - 9/300647710720*(64*(16*(32*x + 81)*x + 1575)*x - 91875)*(32*(32*x + 27)*x - 4096*x^2 - 22464*x + 61425) + 3/150323855360*(32*(32*(32*x + 81)*x + 7875)*x - 160*(16*(32*x + 81)*x + 1575)*x + 3981600*x - 1088325)*(1024*x^2 + 4320*x - 7875) + 27/751619276800*(64*(16*(32*x + 81)*x + 1575)*x - 856800*x + 289275)*(1024*x^2 + 4320*x - 7875) + 243/300647710720*(64*(16*(32*x + 81)*x + 1575)*x - 91875)*(1024*x^2 + 4320*x - 7875) + 9/187904819200*(32*(32*(32*x + 81)*x + 7875)*x - 160*(16*(32*x + 81)*x + 1575)*x + 3981600*x - 1088325)*(1024*x^2 + 864*x - 1575) - 243/375809638400*(64*(16*(32*x + 81)*x + 1575)*x - 856800*x + 289275)*(1024*x^2 + 864*x - 1575) + 2187/1503238553600*(64*(16*(32*x + 81)*x + 1575)*x - 91875)*(1024*x^2 + 864*x - 1575) - 75/8589934592*(2*(96*x - 175)*(32*x + 27) + 192*(192*x - 175)*x - 160*(96*x - 175)*x - 73728

In [141]:
chiang_ogw_a = ChiangOGWInvariant()
chiang_ogw_c = ChiangOGWInvariant(use_recursion_a=False)
# experiment k > 0

for b in range(1, 5):
    for k in range(0, 7):
        for l2 in range(0, 6):
            A1 = 2
            A2 = 3
        
            l1 = get_l1(3, b, k, l2, A1, A2)
            # l1 = 1
            if l1 < 0 :
                continue
            A = [A1]*l1 + [A2]*l2
            chiang_a = chiang_ogw_a.chiang_ogw_invariant(3, b, k, ClassList.compress(A, 3))
            chiang_c = chiang_ogw_c.chiang_ogw_invariant(3, b, k, ClassList.compress(A, 3))

            print(f"b:{b}, k:{k}, A:{A}, chiang_a: {chiang_a}, chiang_c: {chiang_c}")

b:1, k:0, A:[2], chiang_a: x, chiang_c: x
b:1, k:1, A:[], chiang_a: 3, chiang_c: 3
b:2, k:0, A:[2, 2], chiang_a: -175/512, chiang_c: -175/512
b:2, k:0, A:[3], chiang_a: 3/8*x - 175/256, chiang_c: 3/8*x - 175/256
b:2, k:1, A:[2], chiang_a: -9/16, chiang_c: -9/16
b:2, k:2, A:[], chiang_a: 2, chiang_c: 2
b:3, k:0, A:[2, 2, 2], chiang_a: -9/128*x + 2625/8192, chiang_c: -9/128*x + 2625/8192
b:3, k:0, A:[2, 3], chiang_a: -9/64*x + 525/2048, chiang_c: -9/64*x + 525/2048
b:3, k:1, A:[2, 2], chiang_a: 1/8*x + 27/256, chiang_c: 0
b:3, k:1, A:[3], chiang_a: 1/2*x, chiang_c: 1/2*x
b:3, k:2, A:[2], chiang_a: 3/8, chiang_c: 3/8
b:3, k:3, A:[], chiang_a: -128/175*x - 324/175, chiang_c: -216/175
b:4, k:0, A:[2, 2, 2, 2], chiang_a: -1/4096*(32*x + 27)*x + 1/32*x^2 + 351/2048*x - 61425/131072, chiang_c: 1/32*x^2 + 351/2048*x - 61425/131072
b:4, k:0, A:[2, 2, 3], chiang_a: 1/32*x^2 + 135/1024*x - 7875/32768, chiang_c: 1/32*x^2 + 135/1024*x - 7875/32768
b:4, k:0, A:[3, 3], chiang_a: 1/8*x^2 + 27/256*x - 1

In [142]:
x, y = var('x, y')
solve([1/8*x + 27/256 == 0
      ],
      x)

#-1/4*x + 27/256, chiang_c: 525/4096



[x == (-27/32)]

In [143]:
x, y = var('x, y')
solve([-128/175*x - 324/175 == -216/175
      ],
      x)



[x == (-27/32)]

In [144]:
x, y = var('x, y')
solve([ -1/4096*(32*x + 27)*x + 1/32*x^2 + 351/2048*x - 61425/131072 == 1/32*x^2 + 351/2048*x - 61425/131072
      ],
      x)
#1/4096*(64*x - 27)*x + 351/2048*x - 61425/131072, chiang_c: 10707/65536*x - 61425/131072

[x == (-27/32), x == 0]

In [149]:
x, y = var('x, y')
solve([  -15/32*x + 969/2048 == 0],
      x)


[x == (323/320)]

In [146]:
x, y = var('x, y')
solve([ 135/1024*x - 7875/32768 == 135/1024*x - 7875/32768
      ],
      x)

#chiang_a: 135/1024*x - 7875/32768, chiang_c: 135/1024*x - 7875/32768

[x == r1]

In [147]:
b:4, k:1, A:[2, 3],
            chiang_a: -3/16*x + 175/512,
                chiang_c: 81/1024*x - 4725/32768

SyntaxError: invalid syntax (<ipython-input-147-5e7567b4f044>, line 1)